In [ ]:
import pandas as pd
from scipy.cluster.hierarchy import linkage
import numpy as np
import random
import string
import rtsvg
rt = rtsvg.RACETrack()
def randomString(): return ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))
_tiles_, _dfs_ = [], []
for _number_of_models_ in range(5,8):
    for _question_id_ in range(1, 4): 
        for _number_of_scus_ in range(10, 40, 4): # range(4, 100, 4):
            _qid_  = f'q{_question_id_}_m{_number_of_models_}_scu{_number_of_scus_}'
            _scus_ = []
            for i in range(_number_of_scus_): _scus_.append(randomString())
            _lu_ = {'qid':[], 'q':[], 'm':[], 'scu':[]}
            for _model_no_ in range(_number_of_models_):
                num_of_scus = random.randint(1, len(_scus_))
                _scus_choices_ = random.sample(_scus_, num_of_scus)
                for _scu_ in _scus_choices_:
                    _lu_['qid'].append(_qid_)
                    _lu_['q']  .append('???')
                    _lu_['m']  .append(f'm{_model_no_}')
                    _lu_['scu'].append(_scu_)
            df_test  = pd.DataFrame(_lu_)
            _dfs_.append(df_test)
            _list_   = list(set(df_test['m']))
            #
            # Colors are from Colorgorical
            # http://vrl.cs.brown.edu/color
            #@article{gramazio-2017-ccd,
            # author={Gramazio, Connor C. and Laidlaw, David H. and Schloss, Karen B.},
            # journal={IEEE Transactions on Visualization and Computer Graphics},
            # title={Colorgorical: creating discriminable and preferable color palettes for information visualization},
            # year={2017}}
            #
            _colors_ = ["#0cc0aa", "#1d7583", "#68c3ef", "#4149a3", "#ffa8ff", "#ae4acd", "#8f8ac5", "#7b2750", "#f0348f"]
            for i in range(len(_list_)): rt.co_mgr.str_to_color_lu[_list_[i]] = _colors_[i]
            scu_pyramid_test = rtsvg.SCUPyramidMethodDiagram(rt, df_test, 'qid', 'scu', 'm', q_id_multiple=1.2)
            _tiles_.append(scu_pyramid_test.svgSnowman(_qid_, r_scu=14))
            _tiles_.append(scu_pyramid_test.svgCairn  (_qid_, attach_histogram_to_levels=False, rx=0, histogram=True))
            _tiles_.append(scu_pyramid_test.svgPyramid(_qid_))
rt.table(_tiles_, per_row=3, spacer=10)

In [ ]:
# Order Sets By Similarity Example
from scipy.cluster.hierarchy import linkage
import numpy as np
_list_ = [
    {1,2,3},
    {1,3},
    {4,5,6},
    {1,2,3},
    {4,5,8},
    {1,2},
    {1,3}
]
_array_ = []
for i in range(len(_list_)):
    _row_ = []
    for j in range(len(_list_)):
        _similarity_ = len(_list_[i] & _list_[j]) / len(_list_[i] | _list_[j])
        _row_.append(_similarity_)
    _array_.append(_row_)
# Perform hierarchical clustering
linkage_matrix = linkage(_array_, method='ward')
# Place into a tree
parent_to_children = {}
next_node_id       = len(_list_)
for row in linkage_matrix:
    to_merge_0, to_merge_1 = int(row[0]), int(row[1])
    parent_to_children[next_node_id] = [to_merge_0, to_merge_1]
    next_node_id += 1
root_node = next_node_id - 1
# Walk the leaves of the dendrogram
def leafWalk(node_id):
    if node_id < len(_list_):
        return [node_id]
    left_child, right_child = parent_to_children[node_id]
    return leafWalk(left_child) + leafWalk(right_child)
order = leafWalk(root_node)
#for i in order:print(_list_[i])

In [ ]:
q_id_field   = 'qid'
source_field = 'm'
scu_field    = 'scu'

def __scu_level_source_counds__      (df, qid, q_id_field, source_field, scu_field):
    _df_ =  df .query(f'`{q_id_field}` == @qid')
    _df_ = _df_.groupby(scu_field)[source_field].nunique().reset_index().rename({source_field:'__source_count__'},axis=1).sort_values('__source_count__', ascending=False)
    _df_ = _df_.groupby('__source_count__')[scu_field].nunique().reset_index().rename({scu_field:'__scus_on_level__'},axis=1)
    r = {}
    for i in range(len(_df_)):
        scus_on_level = _df_.iloc[i]['__scus_on_level__']
        level         = _df_.iloc[i]['__source_count__']
        r[level] = scus_on_level
    return r

def pyramidTopHeavyScore   (df, qid, q_id_field, source_field, scu_field):
    pass

def pyramidBottomHeavyScore(df, qid, q_id_field, source_field, scu_field):
    pass


In [ ]:
i     = 1
qid   = _dfs_[i].iloc[0][q_id_field]
_svg_ = rtsvg.SCUPyramidMethodDiagram(rt, _dfs_[i], q_id_field, scu_field, source_field).svgCairn(qid)
rt.tile([_svg_])

In [ ]:
__scu_level_source_counds__(_dfs_[i], qid, q_id_field, source_field, scu_field)